In [ ]:
# https://github.com/Cospel/kaggle-rossmann/blob/master/src/data/load_data_hdf5.py
# http://cospel.github.io/notes/Rossmann-Kaggle-Competition/

In [6]:
# Author: Michal Lukac, cospelthetraceur@gmail.com
# script for geting hdf5 dataset for rossmann kaggle competition
# You need to have pandas, numpy, sklearn, ...
# Feel free to contact me!

from pandas import HDFStore
import pandas as pd
import numpy as np
from math import ceil
from sklearn import preprocessing

def get_mean_dataframe(dft, dfs, column='Sales'):
    """
    Get features of mean for every store.
    """
    stores = dfs['Store'].unique()
    days = dft['DayOfWeek'].unique()
    months = dft['Month'].unique()
    state_holidays = dft['StateHoliday'].unique()

    # TODO Mean By StoreType
    # TODO Mean By Assortment
    # TODO Mean By WeekOfMonth
    # TODO Mean By ...

    # create empty arrays
    mean_sales = []
    mean_sales_promo = []
    mean_sales_not_promo = []
    mean_sales_days = { k: [] for k in days }
    mean_sales_months = { k: [] for k in months }
    mean_sales_holiday = { k: [] for k in state_holidays }

    # for every store we get mean value of sales(entire, DayOfWeek, Month)
    for store in stores:
        serie = dft[dft['Store'] == store]

        # entire data mean
        mean_sales.append(serie[column].mean())

        # mean sales by promo
        mean_sales_promo.append(serie[serie['Promo'] == 1][column].mean())
        mean_sales_not_promo.append(serie[serie['Promo'] == 0][column].mean())

        # specific mean by datetime, holidays
        for day in days:
            mean_sales_days[day].append(serie[serie['DayOfWeek'] == day][column].mean())
        for month in months:
            mean_sales_months[month].append(serie[serie['Month'] == month][column].mean())
        for holiday in state_holidays:
            mean_sales_holiday[holiday].append(serie[serie['StateHoliday'] == holiday][column].mean())

    # create dataframes
    df = pd.DataFrame({'Store':                       stores,
                       'Mean' + column:               mean_sales,
                       'Mean' + column + 'Promo' :    mean_sales_promo,
                       'Mean' + column + 'NotPromo' : mean_sales_not_promo})

    # we need to rename_dictionary 1 => MeanDayOfWeek1
    mean_sales_days = rename_dictionary(mean_sales_days, 'MeanDayOfWeek' + column)
    mean_sales_days['Store'] = stores
    df_days = pd.DataFrame(mean_sales_days)

    mean_sales_months = rename_dictionary(mean_sales_months, 'MeanMonth' + column)
    mean_sales_months['Store'] = stores
    df_months = pd.DataFrame(mean_sales_months)

    mean_sales_holiday = rename_dictionary(mean_sales_holiday, 'MeanHoliday' + column)
    mean_sales_holiday['Store'] = stores
    df_holidays = pd.DataFrame(mean_sales_holiday)

    # and normalize
    min_max_scaler = preprocessing.MinMaxScaler()
    df['Mean' + column] = min_max_scaler.fit_transform(df['Mean' + column])
    df['Mean' + column + 'Promo'] = min_max_scaler.fit_transform(df['Mean' + column + 'Promo'])
    df['Mean' + column + 'NotPromo'] = min_max_scaler.fit_transform(df['Mean' + column + 'NotPromo'])

    for day in days:
        df_days['MeanDayOfWeek'+ column + str(day)] = min_max_scaler.fit_transform(df_days['MeanDayOfWeek' + column + str(day)])

    for month in months:
        df_months['MeanMonth' + column + str(month)] = min_max_scaler.fit_transform(df_months['MeanMonth' + column + str(month)])

    for holiday in state_holidays:
        df_holidays['MeanHoliday' + column + str(holiday)] = min_max_scaler.fit_transform(df_holidays['MeanHoliday' + column + str(holiday)])

    # merge everything together on Store
    return pd.merge(df, pd.merge(df_days, pd.merge(df_holidays, df_months, on='Store'), on='Store'), on='Store')

def rename_dictionary(dictionary, name):
    """
    Rename dictionary because dictionary of mean values for month is like { [1-12]: mean }
    and we need { [1-12]MonthMean: mean }
    """
    keys = dictionary.keys()
    for key in keys:
        dictionary[name+str(key)] = dictionary.pop(key)
    return dictionary

def load_data_file(filename,dtypes,parsedate = True):
    """
    Load file to dataframe.
    """
    date_parse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
    if parsedate:
        return pd.read_csv(filename, sep=',', parse_dates=['Date'], date_parser= date_parse,dtype=dtypes)
    else:
        return pd.read_csv(filename, sep=',', dtype=dtypes)

def week_of_month(dt):
    """
    Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

def replace_values(dataframe, column, dictionary):
    """
    Replace values of dataframe column with dictionary values.
    """
    return dataframe[column].apply( lambda x: int(dictionary[x]) )


In [9]:
# some known things about dataset
StateHoliday = {'a': 1, 'b': 2, 'c': 3, '0': 0, 0: 0}
Assortment = {'a': 0, 'b': 1, 'c': 2}
StoreType = {'a': 0,'b': 1,'c': 2, 'd': 3}
Year = { '2013': 1, '2014': 2, '2015': 3 }

# Load data, parse data, clean unwanted columns, replace nan values, create column
print('Loading data ...')
data_dir = '../data/'
data_train = load_data_file(data_dir + 'train.csv',
                            {'Id':np.int32,
                             'Store':np.int32,
                             'DayOfWeek':np.int8,
                             'Sales':np.int32,
                             'Customers':np.int32,
                             'Open':np.int8,
                             'Promo':np.int8,
                             'StateHoliday':np.object, # categorical
                             'SchoolHoliday':np.int8})

data_test = load_data_file(data_dir + 'test.csv',
                            {'Id':np.int32,
                             'Store':np.int32,
                             'DayOfWeek':np.int8,
                             'Open':np.object,         # there is some nan values
                             'Promo':np.int8,
                             'StateHoliday':np.object, # categorical
                             'SchoolHoliday':np.int8})

data_store = load_data_file(data_dir + 'store.csv',
                            {'Store':np.int32,
                             'StoreType':np.object,
                             'Assortment':np.object,
                             'CompetitionDistance':np.float32,
                             'CompetitionOpenSiceMonth':np.object, # categorical
                             'CompetitionOpenSiceYear':np.object,
                             'Promo2':np.int8,
                             'Promo2SinceWeek':np.object,
                             'Promo2SinceYear':np.object,
                             'PromoInterval':np.object}, False)

print('Add some more features ...')
# we have dayofweek already
data_train['Year'] = data_train['Date'].apply(lambda x: str(x)[:4])
data_train['Year'] = replace_values(data_train, 'Year', Year).astype(np.int8)
data_train['Month'] = data_train['Date'].apply(lambda x: int(str(x)[5:7]))
data_train['WeekOfMonth'] = data_train['Date'].apply(lambda x: int(week_of_month(x)))
data_test['Year'] = data_test['Date'].apply(lambda x: str(x)[:4])
data_test['Month'] = data_test['Date'].apply(lambda x: int(str(x)[5:7]))
data_test['WeekOfMonth'] = data_test['Date'].apply(lambda x: int(week_of_month(x)))
data_test['Year'] = replace_values(data_test, 'Year', Year).astype(np.int8)

Loading data ...
Add some more features ...


In [11]:
print('Replacing values as ...')
# categorical values to binary vectors for neural network
data_train = pd.concat([data_train, pd.core.reshape.get_dummies(data_train['StateHoliday'],dummy_na=True, prefix='StateHoliday')],axis=1)
data_test = pd.concat([data_test, pd.get_dummies(data_test['StateHoliday'],dummy_na=True,prefix='StateHoliday')],axis=1)
data_train = pd.concat([data_train, pd.core.reshape.get_dummies(data_train['WeekOfMonth'],dummy_na=True, prefix='WeekOfMonth')],axis=1)
data_test = pd.concat([data_test, pd.core.reshape.get_dummies(data_test['WeekOfMonth'],dummy_na=True, prefix='WeekOfMonth')],axis=1)
data_train = pd.concat([data_train, pd.core.reshape.get_dummies(data_train['Year'],dummy_na=True, prefix='Year')],axis=1)
data_test = pd.concat([data_test, pd.core.reshape.get_dummies(data_test['Year'],dummy_na=True, prefix='Year')],axis=1)
data_train = pd.concat([data_train, pd.core.reshape.get_dummies(data_train['Month'],dummy_na=True, prefix='Month')],axis=1)
data_test = pd.concat([data_test, pd.core.reshape.get_dummies(data_test['Month'],dummy_na=True, prefix='Month')],axis=1)
data_train = pd.concat([data_train, pd.core.reshape.get_dummies(data_train['DayOfWeek'],dummy_na=True, prefix='DayOfWeek')],axis=1)
data_test = pd.concat([data_test, pd.core.reshape.get_dummies(data_test['DayOfWeek'],dummy_na=True, prefix='DayOfWeek')],axis=1)


# in datatest column some columns is missing so we need to add it with default 0
for column in data_train.columns.values.tolist():
    if column not in data_test.columns.values.tolist():
        data_test[column] = 0

data_store = pd.concat([data_store, pd.core.reshape.get_dummies(data_store['Assortment'],dummy_na=True, prefix='Assortment')],axis=1)
data_store = pd.concat([data_store, pd.core.reshape.get_dummies(data_store['StoreType'],dummy_na=True, prefix='StoreType')],axis=1)

# this is simple converting to integer
data_train['StateHoliday'] = replace_values(data_train,'StateHoliday', StateHoliday).astype(np.int8)
data_test['StateHoliday'] = replace_values(data_test,'StateHoliday', StateHoliday).astype(np.int8)
data_store['Assortment'] = replace_values(data_store,'Assortment', Assortment).astype(np.int8)
data_store['StoreType'] = replace_values(data_store,'StoreType', StoreType).astype(np.int8)

# create mean dataframe
print('Mean data frame features ...')
data_mean1 = get_mean_dataframe(data_train, data_store, 'Sales')
# data_mean2 = get_mean_dataframe(data_train, data_store, 'Customers')
# data_mean = pd.merge(data_mean1, data_mean2, on='Store')
data_mean1.head()

Replacing values as ...
Mean data frame features ...


,MeanSales,MeanSalesNotPromo,MeanSalesPromo,Store,MeanDayOfWeekSales1,MeanDayOfWeekSales2,MeanDayOfWeekSales3,MeanDayOfWeekSales4,MeanDayOfWeekSales5,MeanDayOfWeekSales6,...,MeanMonthSales11,MeanMonthSales12,MeanMonthSales2,MeanMonthSales3,MeanMonthSales4,MeanMonthSales5,MeanMonthSales6,MeanMonthSales7,MeanMonthSales8,MeanMonthSales9
0,0.092086,0.099002,0.092153,1,0.088608,0.093604,0.100245,0.086218,0.101748,0.190830,...,0.115818,0.104830,0.109688,0.105413,0.084969,0.085138,0.105861,0.108197,0.110705,0.118461
1,0.101683,0.081142,0.139573,2,0.128607,0.126285,0.163712,0.116937,0.098828,0.092264,...,0.118437,0.103668,0.107860,0.111075,0.094634,0.089988,0.128770,0.127368,0.125128,0.148556
2,0.189279,0.138915,0.252822,3,0.231611,0.235081,0.228071,0.207112,0.225516,0.166623,...,0.198343,0.194021,0.196792,0.199970,0.178391,0.166268,0.220317,0.224442,0.206590,0.230602
3,0.312724,0.273999,0.334734,4,0.345328,0.319467,0.316769,0.322022,0.345280,0.437044,...,0.339170,0.309388,0.341266,0.319721,0.293769,0.274733,0.327506,0.338340,0.340405,0.347876
4,0.087832,0.066976,0.128947,5,0.130708,0.113879,0.128892,0.104104,0.106569,0.054455,...,0.101435,0.097513,0.090288,0.086236,0.079492,0.079973,0.112058,0.123761,0.116873,0.131185


In [12]:
data_mean1.columns

Index([u'MeanSales', u'MeanSalesNotPromo', u'MeanSalesPromo', u'Store',
       u'MeanDayOfWeekSales1', u'MeanDayOfWeekSales2', u'MeanDayOfWeekSales3',
       u'MeanDayOfWeekSales4', u'MeanDayOfWeekSales5', u'MeanDayOfWeekSales6',
       u'MeanDayOfWeekSales7', u'MeanHolidaySales0', u'MeanHolidaySales1',
       u'MeanHolidaySales2', u'MeanHolidaySales3', u'MeanMonthSales1',
       u'MeanMonthSales10', u'MeanMonthSales11', u'MeanMonthSales12',
       u'MeanMonthSales2', u'MeanMonthSales3', u'MeanMonthSales4',
       u'MeanMonthSales5', u'MeanMonthSales6', u'MeanMonthSales7',
       u'MeanMonthSales8', u'MeanMonthSales9'],
      dtype='object')